# 1. Import Libraries

In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Features to be Used

Below are the preliminary features that will be used to build the model. More will be added on afterwards.

**Fatigue**
- Days since Home Team played last match 
- Days since Away Team played last match

**Home Team Form**
- Goals difference of home team in the last x matches    
- Goals difference of home team in the last x home matches    
- Average number of points gained by home team in the last x matches
- Average number of points gained by home team in the last x home matches
- Home Team Win streak 

**Away Team Form**
- Goals difference of away team in the last x matches  
- Goals difference of away team in the last x away matches
- Average number of points gained by away team in the last x matches
- Average number of points gained by away team in the last x away matches
- Away Team Win streak

**Home Team Performance Index**
- Home Defense Performance Index
- Home Midfield Performance Index
- Home Attack Performance Index

**Away Team Performance Index**
- Away Defense Performance Index
- Away Midfield Performance Index
- Away Attack Performance Index

**Betting Odds**
- B365H
- B365D
- B365A

In [5]:
df = pd.read_csv('datasets/integrated_dataset.csv')
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50,D,NaN,7,10,NaN,NaN
1,2012-08-18,fulham,norwich,5.0,0.0,11.0,4.0,9.0,2.0,12.0,11.0,6.0,3.0,0.0,0.0,0.0,0.0,1.80,3.60,4.50,W,NaN,10,10,NaN,NaN
2,2012-08-18,newcastle,tottenham,2.0,1.0,6.0,12.0,4.0,6.0,12.0,8.0,3.0,5.0,2.0,2.0,0.0,0.0,2.50,3.40,2.75,W,NaN,10,10,NaN,NaN
3,2012-08-18,qpr,swansea,0.0,5.0,20.0,12.0,11.0,8.0,11.0,14.0,5.0,3.0,2.0,2.0,0.0,0.0,2.00,3.40,3.80,L,NaN,10,10,NaN,NaN
4,2012-08-18,reading,stoke,1.0,1.0,9.0,6.0,3.0,3.0,9.0,14.0,4.0,3.0,2.0,4.0,0.0,1.0,2.38,3.25,3.10,D,NaN,10,10,NaN,NaN


In [6]:
df.shape

(3800, 26)

# 3. EDA

In [7]:
# # 1. Get initial insight into the data
# df.info()

# # 2. View qualitative measures of the data. 
# # Make sense of the minimum and maximum values of certain columns. Plot a graph.
# df.describe()

# # 3. Removing Redundant variables

# # 4. Check for missing values
# df.isnull().sum()

# # 5. Correlation
# plt.figure(figsize=(10,10))
# sns.heatmap(auto.corr(),cbar=True,annot=True,cmap='Blues')

# # 6. create scatterplots between all of your variables
# sns.pairplot(df_cleaned)

# # 7. Analyze relationships between an interesting pair of variables 
# # (How does the horsepower affect the price?)
# #
# plt.figure(figsize=(10,10))
# plt.scatter(x='horsepower',y='price',data=auto)
# plt.xlabel('Horsepower')
# plt.ylabel('Price')
# 
# sns.histplot(auto.horsepower,bins=10)

# # 8. Use boxplot to detect outliers
# sns.boxplot(x='price',y='num_of_doors',data=auto)

# # 9. Normalize the data (through normalizing, scaling, or standardizing the data)
# 

# 4. Feature Engineering

## Goals difference of home team in the last x matches

In [44]:
def getGoalsScoredByTeam(row, team):
    if row['HomeTeam'] == team:
        return row['FTHG']
    elif row['AwayTeam'] == team:
        return row['FTAG']

In [45]:
def getGoalsScoredByOpponent(row, team):
    if row['HomeTeam'] == team:
        return row['FTAG']
    elif row['AwayTeam'] == team:
        return row['FTHG']

In [46]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    team_df = team_df.sort_values(by = 'Date')
    
    number_of_previous_matches = 5
    column1 = 0;
    column2 = 0;
    
    team_df['GoalsScoredByTeam'] = team_df.apply(lambda row: getGoalsScoredByTeam(row, team), axis=1)
    team_df['GoalsScoredByOpponent'] = team_df.apply(lambda row: getGoalsScoredByOpponent(row, team), axis=1)
    
    for i in range(1, number_of_previous_matches+1):
        column1 += team_df['GoalsScoredByTeam'].shift(i)
        column2 += team_df['GoalsScoredByOpponent'].shift(i)
    
    team_df['HGDL5M'] = column1 - column2
    team_df.drop(columns=['GoalsScoredByTeam', 'GoalsScoredByOpponent'], inplace=True)
    team_df = team_df[team_df['HomeTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [47]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.5,D,NaN,7,10,NaN,NaN,NaN
1,2012-09-15,arsenal,southampton,6.0,1.0,20.0,9.0,12.0,4.0,4.0,7.0,8.0,3.0,0.0,0.0,0.0,0.0,1.36,5.00,8.5,W,NaN,10,10,NaN,NaN,NaN
2,2012-09-29,arsenal,chelsea,1.0,2.0,14.0,10.0,5.0,5.0,14.0,13.0,6.0,5.0,2.0,3.0,0.0,0.0,2.10,3.40,3.5,L,NaN,6,7,NaN,NaN,7.0
3,2012-10-27,arsenal,qpr,1.0,0.0,21.0,4.0,11.0,3.0,14.0,9.0,5.0,2.0,1.0,2.0,0.0,1.0,1.36,5.00,8.5,W,NaN,3,6,NaN,NaN,5.0
4,2012-11-10,arsenal,fulham,3.0,3.0,15.0,9.0,9.0,5.0,12.0,10.0,8.0,6.0,1.0,2.0,0.0,0.0,1.53,4.20,6.0,D,NaN,4,7,NaN,NaN,0.0
5,2012-11-17,arsenal,tottenham,5.0,2.0,13.0,6.0,10.0,4.0,13.0,12.0,6.0,6.0,1.0,2.0,0.0,1.0,1.91,3.75,3.8,W,NaN,7,6,NaN,NaN,1.0
6,2012-12-01,arsenal,swansea,0.0,2.0,10.0,12.0,8.0,10.0,11.0,9.0,8.0,5.0,1.0,0.0,0.0,0.0,1.44,4.50,7.0,L,NaN,3,3,NaN,NaN,2.0
7,2012-12-08,arsenal,west-brom,2.0,0.0,16.0,7.0,7.0,2.0,11.0,12.0,7.0,4.0,2.0,3.0,0.0,0.0,1.50,4.33,6.5,W,NaN,4,7,NaN,NaN,1.0
8,2012-12-29,arsenal,newcastle,7.0,3.0,16.0,10.0,12.0,8.0,8.0,10.0,5.0,6.0,0.0,1.0,0.0,0.0,1.44,4.50,7.0,W,NaN,7,3,NaN,NaN,4.0
9,2013-01-13,arsenal,city,0.0,2.0,9.0,16.0,4.0,9.0,11.0,18.0,2.0,7.0,2.0,4.0,1.0,1.0,2.63,3.50,2.8,L,NaN,10,10,NaN,NaN,10.0


In [48]:
df.shape

(3800, 27)

## Goals difference of home team in the last x home matches

In [49]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[df['HomeTeam'] == team]
    
    number_of_previous_matches = 5
    column1 = 0;
    column2 = 0
    
    team_df = team_df.sort_values(by = 'Date')
    for i in range(1, number_of_previous_matches+1):
        column1 += team_df['FTHG'].shift(i)
        column2 += team_df['FTAG'].shift(i)

    team_df['HGDL5HM'] = column1 - column2
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [50]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.5,D,NaN,7,10,NaN,NaN,NaN,NaN
1,2012-09-15,arsenal,southampton,6.0,1.0,20.0,9.0,12.0,4.0,4.0,7.0,8.0,3.0,0.0,0.0,0.0,0.0,1.36,5.00,8.5,W,NaN,10,10,NaN,NaN,NaN,NaN
2,2012-09-29,arsenal,chelsea,1.0,2.0,14.0,10.0,5.0,5.0,14.0,13.0,6.0,5.0,2.0,3.0,0.0,0.0,2.10,3.40,3.5,L,NaN,6,7,NaN,NaN,7.0,NaN
3,2012-10-27,arsenal,qpr,1.0,0.0,21.0,4.0,11.0,3.0,14.0,9.0,5.0,2.0,1.0,2.0,0.0,1.0,1.36,5.00,8.5,W,NaN,3,6,NaN,NaN,5.0,NaN
4,2012-11-10,arsenal,fulham,3.0,3.0,15.0,9.0,9.0,5.0,12.0,10.0,8.0,6.0,1.0,2.0,0.0,0.0,1.53,4.20,6.0,D,NaN,4,7,NaN,NaN,0.0,NaN
5,2012-11-17,arsenal,tottenham,5.0,2.0,13.0,6.0,10.0,4.0,13.0,12.0,6.0,6.0,1.0,2.0,0.0,1.0,1.91,3.75,3.8,W,NaN,7,6,NaN,NaN,1.0,5.0
6,2012-12-01,arsenal,swansea,0.0,2.0,10.0,12.0,8.0,10.0,11.0,9.0,8.0,5.0,1.0,0.0,0.0,0.0,1.44,4.50,7.0,L,NaN,3,3,NaN,NaN,2.0,8.0
7,2012-12-08,arsenal,west-brom,2.0,0.0,16.0,7.0,7.0,2.0,11.0,12.0,7.0,4.0,2.0,3.0,0.0,0.0,1.50,4.33,6.5,W,NaN,4,7,NaN,NaN,1.0,1.0
8,2012-12-29,arsenal,newcastle,7.0,3.0,16.0,10.0,12.0,8.0,8.0,10.0,5.0,6.0,0.0,1.0,0.0,0.0,1.44,4.50,7.0,W,NaN,7,3,NaN,NaN,4.0,4.0
9,2013-01-13,arsenal,city,0.0,2.0,9.0,16.0,4.0,9.0,11.0,18.0,2.0,7.0,2.0,4.0,1.0,1.0,2.63,3.50,2.8,L,NaN,10,10,NaN,NaN,10.0,7.0


In [51]:
df.shape

(3800, 28)

## Average number of points gained by home team in the last x matches

In [52]:
def getPointsGained(row, team):
    if row['HomeTeam'] == team:
        if row['Result'] == 'W':
            return 3
        elif row['Result'] == 'D':
            return 1
        elif row['Result'] == 'L':
            return 0
    else:
        if row['Result'] == 'W':
            return 0
        elif row['Result'] == 'D':
            return 1
        elif row['Result'] == 'L':
            return 3

In [53]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    team_df = team_df.sort_values(by = 'Date')
    
    team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row, team), axis=1)
    
    number_of_previous_matches = 5
    column = 0
    
    
    for i in range(1, number_of_previous_matches+1):
        column += team_df['Points Gained'].shift(i)
    
    team_df['HAPL5M'] = column / number_of_previous_matches
    team_df.drop(columns=['Points Gained'], inplace=True)
    
    team_df = team_df[team_df['HomeTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [54]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.5,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN
1,2012-09-15,arsenal,southampton,6.0,1.0,20.0,9.0,12.0,4.0,4.0,7.0,8.0,3.0,0.0,0.0,0.0,0.0,1.36,5.00,8.5,W,NaN,10,10,NaN,NaN,NaN,NaN,NaN
2,2012-09-29,arsenal,chelsea,1.0,2.0,14.0,10.0,5.0,5.0,14.0,13.0,6.0,5.0,2.0,3.0,0.0,0.0,2.10,3.40,3.5,L,NaN,6,7,NaN,NaN,7.0,NaN,1.8
3,2012-10-27,arsenal,qpr,1.0,0.0,21.0,4.0,11.0,3.0,14.0,9.0,5.0,2.0,1.0,2.0,0.0,1.0,1.36,5.00,8.5,W,NaN,3,6,NaN,NaN,5.0,NaN,1.4
4,2012-11-10,arsenal,fulham,3.0,3.0,15.0,9.0,9.0,5.0,12.0,10.0,8.0,6.0,1.0,2.0,0.0,0.0,1.53,4.20,6.0,D,NaN,4,7,NaN,NaN,0.0,NaN,1.2
5,2012-11-17,arsenal,tottenham,5.0,2.0,13.0,6.0,10.0,4.0,13.0,12.0,6.0,6.0,1.0,2.0,0.0,1.0,1.91,3.75,3.8,W,NaN,7,6,NaN,NaN,1.0,5.0,1.4
6,2012-12-01,arsenal,swansea,0.0,2.0,10.0,12.0,8.0,10.0,11.0,9.0,8.0,5.0,1.0,0.0,0.0,0.0,1.44,4.50,7.0,L,NaN,3,3,NaN,NaN,2.0,8.0,1.2
7,2012-12-08,arsenal,west-brom,2.0,0.0,16.0,7.0,7.0,2.0,11.0,12.0,7.0,4.0,2.0,3.0,0.0,0.0,1.50,4.33,6.5,W,NaN,4,7,NaN,NaN,1.0,1.0,1.2
8,2012-12-29,arsenal,newcastle,7.0,3.0,16.0,10.0,12.0,8.0,8.0,10.0,5.0,6.0,0.0,1.0,0.0,0.0,1.44,4.50,7.0,W,NaN,7,3,NaN,NaN,4.0,4.0,2.0
9,2013-01-13,arsenal,city,0.0,2.0,9.0,16.0,4.0,9.0,11.0,18.0,2.0,7.0,2.0,4.0,1.0,1.0,2.63,3.50,2.8,L,NaN,10,10,NaN,NaN,10.0,7.0,2.6


In [55]:
df.shape

(3800, 29)

## Average number of points gained by home team in the last x home matches

In [56]:
def getPointsGained(row):
    if row['Result'] == 'W':
        return 3
    elif row['Result'] == 'D':
        return 1
    elif row['Result'] == 'L':
        return 0

In [57]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[(df['HomeTeam'] == team)]
    team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
    
    number_of_previous_matches = 5
    column = 0
    
    team_df = team_df.sort_values(by = 'Date')
    for i in range(1, number_of_previous_matches+1):
        column += team_df['Points Gained'].shift(i)
    
    team_df['HAPL5HM'] = column / number_of_previous_matches
    team_df.drop(columns=['Points Gained'], inplace=True)
    
    team_df = team_df[team_df['HomeTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1435245815.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1435245815.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1435245815.py:5: SettingWithCopyWarning: 
A value is trying to be set 

C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1435245815.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1435245815.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)


In [58]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M,HAPL5HM
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.5,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-09-15,arsenal,southampton,6.0,1.0,20.0,9.0,12.0,4.0,4.0,7.0,8.0,3.0,0.0,0.0,0.0,0.0,1.36,5.00,8.5,W,NaN,10,10,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-09-29,arsenal,chelsea,1.0,2.0,14.0,10.0,5.0,5.0,14.0,13.0,6.0,5.0,2.0,3.0,0.0,0.0,2.10,3.40,3.5,L,NaN,6,7,NaN,NaN,7.0,NaN,1.8,NaN
3,2012-10-27,arsenal,qpr,1.0,0.0,21.0,4.0,11.0,3.0,14.0,9.0,5.0,2.0,1.0,2.0,0.0,1.0,1.36,5.00,8.5,W,NaN,3,6,NaN,NaN,5.0,NaN,1.4,NaN
4,2012-11-10,arsenal,fulham,3.0,3.0,15.0,9.0,9.0,5.0,12.0,10.0,8.0,6.0,1.0,2.0,0.0,0.0,1.53,4.20,6.0,D,NaN,4,7,NaN,NaN,0.0,NaN,1.2,NaN
5,2012-11-17,arsenal,tottenham,5.0,2.0,13.0,6.0,10.0,4.0,13.0,12.0,6.0,6.0,1.0,2.0,0.0,1.0,1.91,3.75,3.8,W,NaN,7,6,NaN,NaN,1.0,5.0,1.4,1.6
6,2012-12-01,arsenal,swansea,0.0,2.0,10.0,12.0,8.0,10.0,11.0,9.0,8.0,5.0,1.0,0.0,0.0,0.0,1.44,4.50,7.0,L,NaN,3,3,NaN,NaN,2.0,8.0,1.2,2.0
7,2012-12-08,arsenal,west-brom,2.0,0.0,16.0,7.0,7.0,2.0,11.0,12.0,7.0,4.0,2.0,3.0,0.0,0.0,1.50,4.33,6.5,W,NaN,4,7,NaN,NaN,1.0,1.0,1.2,1.4
8,2012-12-29,arsenal,newcastle,7.0,3.0,16.0,10.0,12.0,8.0,8.0,10.0,5.0,6.0,0.0,1.0,0.0,0.0,1.44,4.50,7.0,W,NaN,7,3,NaN,NaN,4.0,4.0,2.0,2.0
9,2013-01-13,arsenal,city,0.0,2.0,9.0,16.0,4.0,9.0,11.0,18.0,2.0,7.0,2.0,4.0,1.0,1.0,2.63,3.50,2.8,L,NaN,10,10,NaN,NaN,10.0,7.0,2.6,2.0


In [59]:
df.shape

(3800, 30)

## Home Team Win streak

## Goals difference of away team in the last x matches  

In [60]:
def getGoalsScoredByTeam(row, team):
    if row['HomeTeam'] == team:
        return row['FTHG']
    elif row['AwayTeam'] == team:
        return row['FTAG']

In [61]:
def getGoalsScoredByOpponent(row, team):
    if row['HomeTeam'] == team:
        return row['FTAG']
    elif row['AwayTeam'] == team:
        return row['FTHG']

In [62]:
temp_df = pd.DataFrame()

for team in df['AwayTeam'].unique():
    team_df = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    team_df = team_df.sort_values(by = 'Date')
    
    number_of_previous_matches = 5
    column1 = 0;
    column2 = 0;
    
    team_df['GoalsScoredByTeam'] = team_df.apply(lambda row: getGoalsScoredByTeam(row, team), axis=1)
    team_df['GoalsScoredByOpponent'] = team_df.apply(lambda row: getGoalsScoredByOpponent(row, team), axis=1)
    
    for i in range(1, number_of_previous_matches+1):
        column1 += team_df['GoalsScoredByTeam'].shift(i)
        column2 += team_df['GoalsScoredByOpponent'].shift(i)
    
    team_df['AGDL5M'] = column1 - column2
    team_df.drop(columns=['GoalsScoredByTeam', 'GoalsScoredByOpponent'], inplace=True)
    team_df = team_df[team_df['AwayTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [63]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M,HAPL5HM,AGDL5M
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-09-01,swansea,sunderland,2.0,2.0,14.0,4.0,10.0,3.0,11.0,7.0,8.0,0.0,1.0,1.0,1.0,0.0,2.20,3.30,3.40,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-09-22,west-ham,sunderland,1.0,1.0,20.0,4.0,11.0,2.0,12.0,10.0,10.0,1.0,2.0,1.0,0.0,0.0,2.10,3.30,3.60,D,NaN,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-10-06,city,sunderland,3.0,0.0,26.0,8.0,16.0,6.0,10.0,10.0,15.0,4.0,0.0,3.0,0.0,0.0,1.29,5.50,11.00,W,NaN,3,7,NaN,NaN,3.0,NaN,1.8,NaN,1.0
4,2012-10-27,stoke,sunderland,0.0,0.0,11.0,6.0,6.0,3.0,12.0,6.0,7.0,3.0,2.0,2.0,0.0,0.0,1.95,3.30,4.20,D,NaN,7,6,NaN,NaN,-1.0,NaN,1.0,NaN,-2.0
5,2012-11-10,everton,sunderland,2.0,1.0,16.0,6.0,10.0,4.0,14.0,14.0,6.0,2.0,1.0,3.0,0.0,0.0,1.44,4.20,8.00,W,NaN,7,7,NaN,NaN,2.0,NaN,1.4,NaN,-3.0
6,2012-11-18,fulham,sunderland,1.0,3.0,18.0,12.0,13.0,9.0,9.0,13.0,4.0,4.0,1.0,3.0,1.0,0.0,1.67,3.75,5.25,L,NaN,8,8,NaN,NaN,1.0,8.0,1.4,2.0,-5.0
7,2012-12-02,norwich,sunderland,2.0,1.0,7.0,16.0,5.0,6.0,11.0,16.0,2.0,6.0,0.0,2.0,0.0,0.0,2.10,3.30,3.60,W,NaN,4,5,NaN,NaN,2.0,0.0,1.8,2.0,-2.0
8,2012-12-15,united,sunderland,3.0,1.0,15.0,13.0,6.0,10.0,13.0,10.0,9.0,9.0,0.0,0.0,0.0,0.0,1.20,6.50,15.00,W,NaN,6,4,NaN,NaN,4.0,5.0,2.4,2.4,-2.0
9,2012-12-22,southampton,sunderland,0.0,1.0,13.0,10.0,7.0,6.0,11.0,15.0,8.0,5.0,0.0,3.0,0.0,0.0,2.00,3.50,3.75,L,NaN,10,7,NaN,NaN,4.0,2.0,2.0,1.6,-2.0


## Goals difference of away team in the last x away matches

In [64]:
temp_df = pd.DataFrame()

for team in df['AwayTeam'].unique():
    team_df = df[df['AwayTeam'] == team]
    team_df = team_df.sort_values(by = 'Date')
    
    number_of_previous_matches = 5
    column1 = 0;
    column2 = 0
    
    for i in range(1, number_of_previous_matches+1):
        column1 += team_df['FTHG'].shift(i)
        column2 += team_df['FTAG'].shift(i)

    team_df['AGDL5AM'] = column2 - column1
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [65]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M,HAPL5HM,AGDL5M,AGDL5AM
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-09-01,swansea,sunderland,2.0,2.0,14.0,4.0,10.0,3.0,11.0,7.0,8.0,0.0,1.0,1.0,1.0,0.0,2.20,3.30,3.40,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-09-22,west-ham,sunderland,1.0,1.0,20.0,4.0,11.0,2.0,12.0,10.0,10.0,1.0,2.0,1.0,0.0,0.0,2.10,3.30,3.60,D,NaN,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-10-06,city,sunderland,3.0,0.0,26.0,8.0,16.0,6.0,10.0,10.0,15.0,4.0,0.0,3.0,0.0,0.0,1.29,5.50,11.00,W,NaN,3,7,NaN,NaN,3.0,NaN,1.8,NaN,1.0,NaN
4,2012-10-27,stoke,sunderland,0.0,0.0,11.0,6.0,6.0,3.0,12.0,6.0,7.0,3.0,2.0,2.0,0.0,0.0,1.95,3.30,4.20,D,NaN,7,6,NaN,NaN,-1.0,NaN,1.0,NaN,-2.0,NaN
5,2012-11-10,everton,sunderland,2.0,1.0,16.0,6.0,10.0,4.0,14.0,14.0,6.0,2.0,1.0,3.0,0.0,0.0,1.44,4.20,8.00,W,NaN,7,7,NaN,NaN,2.0,NaN,1.4,NaN,-3.0,-3.0
6,2012-11-18,fulham,sunderland,1.0,3.0,18.0,12.0,13.0,9.0,9.0,13.0,4.0,4.0,1.0,3.0,1.0,0.0,1.67,3.75,5.25,L,NaN,8,8,NaN,NaN,1.0,8.0,1.4,2.0,-5.0,-4.0
7,2012-12-02,norwich,sunderland,2.0,1.0,7.0,16.0,5.0,6.0,11.0,16.0,2.0,6.0,0.0,2.0,0.0,0.0,2.10,3.30,3.60,W,NaN,4,5,NaN,NaN,2.0,0.0,1.8,2.0,-2.0,-2.0
8,2012-12-15,united,sunderland,3.0,1.0,15.0,13.0,6.0,10.0,13.0,10.0,9.0,9.0,0.0,0.0,0.0,0.0,1.20,6.50,15.00,W,NaN,6,4,NaN,NaN,4.0,5.0,2.4,2.4,-2.0,-3.0
9,2012-12-22,southampton,sunderland,0.0,1.0,13.0,10.0,7.0,6.0,11.0,15.0,8.0,5.0,0.0,3.0,0.0,0.0,2.00,3.50,3.75,L,NaN,10,7,NaN,NaN,4.0,2.0,2.0,1.6,-2.0,-2.0


In [66]:
df.shape

(3800, 32)

## Average number of points gained by away team in the last x matches

In [67]:
def getPointsGained(row, team):
    if row['HomeTeam'] == team:
        if row['Result'] == 'W':
            return 3
        elif row['Result'] == 'D':
            return 1
        elif row['Result'] == 'L':
            return 0
    else:
        if row['Result'] == 'W':
            return 0
        elif row['Result'] == 'D':
            return 1
        elif row['Result'] == 'L':
            return 3

In [68]:
temp_df = pd.DataFrame()

for team in df['AwayTeam'].unique():
    team_df = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    team_df = team_df.sort_values(by = 'Date')
    
    team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row, team), axis=1)
    
    number_of_previous_matches = 5
    column = 0
    
    for i in range(1, number_of_previous_matches+1):
        column += team_df['Points Gained'].shift(i)
    
    team_df['AAPL5M'] = column / number_of_previous_matches
    team_df.drop(columns=['Points Gained'], inplace=True)
    
    team_df = team_df[team_df['AwayTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [69]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M,HAPL5HM,AGDL5M,AGDL5AM,AAPL5M
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-09-01,swansea,sunderland,2.0,2.0,14.0,4.0,10.0,3.0,11.0,7.0,8.0,0.0,1.0,1.0,1.0,0.0,2.20,3.30,3.40,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-09-22,west-ham,sunderland,1.0,1.0,20.0,4.0,11.0,2.0,12.0,10.0,10.0,1.0,2.0,1.0,0.0,0.0,2.10,3.30,3.60,D,NaN,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-10-06,city,sunderland,3.0,0.0,26.0,8.0,16.0,6.0,10.0,10.0,15.0,4.0,0.0,3.0,0.0,0.0,1.29,5.50,11.00,W,NaN,3,7,NaN,NaN,3.0,NaN,1.8,NaN,1.0,NaN,1.4
4,2012-10-27,stoke,sunderland,0.0,0.0,11.0,6.0,6.0,3.0,12.0,6.0,7.0,3.0,2.0,2.0,0.0,0.0,1.95,3.30,4.20,D,NaN,7,6,NaN,NaN,-1.0,NaN,1.0,NaN,-2.0,NaN,1.2
5,2012-11-10,everton,sunderland,2.0,1.0,16.0,6.0,10.0,4.0,14.0,14.0,6.0,2.0,1.0,3.0,0.0,0.0,1.44,4.20,8.00,W,NaN,7,7,NaN,NaN,2.0,NaN,1.4,NaN,-3.0,-3.0,1.0
6,2012-11-18,fulham,sunderland,1.0,3.0,18.0,12.0,13.0,9.0,9.0,13.0,4.0,4.0,1.0,3.0,1.0,0.0,1.67,3.75,5.25,L,NaN,8,8,NaN,NaN,1.0,8.0,1.4,2.0,-5.0,-4.0,0.4
7,2012-12-02,norwich,sunderland,2.0,1.0,7.0,16.0,5.0,6.0,11.0,16.0,2.0,6.0,0.0,2.0,0.0,0.0,2.10,3.30,3.60,W,NaN,4,5,NaN,NaN,2.0,0.0,1.8,2.0,-2.0,-2.0,0.8
8,2012-12-15,united,sunderland,3.0,1.0,15.0,13.0,6.0,10.0,13.0,10.0,9.0,9.0,0.0,0.0,0.0,0.0,1.20,6.50,15.00,W,NaN,6,4,NaN,NaN,4.0,5.0,2.4,2.4,-2.0,-3.0,0.8
9,2012-12-22,southampton,sunderland,0.0,1.0,13.0,10.0,7.0,6.0,11.0,15.0,8.0,5.0,0.0,3.0,0.0,0.0,2.00,3.50,3.75,L,NaN,10,7,NaN,NaN,4.0,2.0,2.0,1.6,-2.0,-2.0,0.8


In [70]:
df.shape

(3800, 33)

## Average number of points gained by away team in the last x away matches

In [71]:
def getPointsGained(row):
    if row['Result'] == 'W':
        return 0
    elif row['Result'] == 'D':
        return 1
    elif row['Result'] == 'L':
        return 3

In [72]:
temp_df = pd.DataFrame()

for team in df['AwayTeam'].unique():
    team_df = df[(df['AwayTeam'] == team)]
    team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
    team_df = team_df.sort_values(by = 'Date')
    
    number_of_previous_matches = 5
    column = 0
    
    for i in range(1, number_of_previous_matches+1):
        column += team_df['Points Gained'].shift(i)
    
    team_df['AAPL5AM'] = column / number_of_previous_matches
    team_df.drop(columns=['Points Gained'], inplace=True)
    
    team_df = team_df[team_df['AwayTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1041028938.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1041028938.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1041028938.py:5: SettingWithCopyWarning: 
A value is trying to be set 

C:\Users\choibs\AppData\Local\Temp\ipykernel_2256\1041028938.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)


In [73]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M,HAPL5HM,AGDL5M,AGDL5AM,AAPL5M,AAPL5AM
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.5,8.5,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-09-01,swansea,sunderland,2.0,2.0,14.0,4.0,10.0,3.0,11.0,7.0,8.0,0.0,1.0,1.0,1.0,0.0,2.20,3.3,3.4,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-09-22,west-ham,sunderland,1.0,1.0,20.0,4.0,11.0,2.0,12.0,10.0,10.0,1.0,2.0,1.0,0.0,0.0,2.10,3.3,3.6,D,NaN,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-10-06,city,sunderland,3.0,0.0,26.0,8.0,16.0,6.0,10.0,10.0,15.0,4.0,0.0,3.0,0.0,0.0,1.29,5.5,11.0,W,NaN,3,7,NaN,NaN,3.0,NaN,1.8,NaN,1.0,NaN,1.4,NaN
4,2012-10-27,stoke,sunderland,0.0,0.0,11.0,6.0,6.0,3.0,12.0,6.0,7.0,3.0,2.0,2.0,0.0,0.0,1.95,3.3,4.2,D,NaN,7,6,NaN,NaN,-1.0,NaN,1.0,NaN,-2.0,NaN,1.2,NaN


In [74]:
df.shape

(3800, 34)

## Away Team Win streak

## Home Team Performance Index
- Home Defense Performance Index
- Home Midfield Performance Index
- Home Attack Performance Index

In [75]:
fifa_ratings_df = pd.read_csv('datasets/data-source-3/dataset3.csv')
fifa_ratings_df.head()

,Version,Date,Team,Attack,Midfield,Defense
0,FIFA 22,2022-08-18,liverpool,86,84,85
1,FIFA 22,2022-08-18,city,84,87,86
2,FIFA 22,2022-08-18,chelsea,84,85,83
3,FIFA 22,2022-08-18,united,82,83,81
4,FIFA 22,2022-08-18,tottenham,83,81,78


In [76]:
def getHomeTeamRatingIndex(row):
    date = row['Date']
    hometeam = row['HomeTeam']
    
    ratings_df = fifa_ratings_df[(fifa_ratings_df['Team'] == hometeam) & (fifa_ratings_df['Date'] < date)]
    ratings_row = ratings_df.sort_values(by = ['Date', 'Version'], ascending=[False, False]).iloc[0, :]
    
    row['HDef'] = ratings_row['Defense']
    row['HMid'] = ratings_row['Midfield']
    row['HAtt'] = ratings_row['Attack']
    
    return row

In [77]:
df = df.apply(getHomeTeamRatingIndex, axis=1)
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M,HAPL5HM,AGDL5M,AGDL5AM,AAPL5M,AAPL5AM,HDef,HMid,HAtt
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.5,8.5,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,82,84
1,2012-09-01,swansea,sunderland,2.0,2.0,14.0,4.0,10.0,3.0,11.0,7.0,8.0,0.0,1.0,1.0,1.0,0.0,2.20,3.3,3.4,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,73,73
2,2012-09-22,west-ham,sunderland,1.0,1.0,20.0,4.0,11.0,2.0,12.0,10.0,10.0,1.0,2.0,1.0,0.0,0.0,2.10,3.3,3.6,D,NaN,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71,73,75
3,2012-10-06,city,sunderland,3.0,0.0,26.0,8.0,16.0,6.0,10.0,10.0,15.0,4.0,0.0,3.0,0.0,0.0,1.29,5.5,11.0,W,NaN,3,7,NaN,NaN,3.0,NaN,1.8,NaN,1.0,NaN,1.4,NaN,81,83,85
4,2012-10-27,stoke,sunderland,0.0,0.0,11.0,6.0,6.0,3.0,12.0,6.0,7.0,3.0,2.0,2.0,0.0,0.0,1.95,3.3,4.2,D,NaN,7,6,NaN,NaN,-1.0,NaN,1.0,NaN,-2.0,NaN,1.2,NaN,75,76,75


## Away Team Performance Index

- Away Defense Performance Index
- Away Midfield Performance Index
- Away Attack Performance Index

In [78]:
def getAwayTeamRatingIndex(row):
    date = row['Date']
    awayteam = row['AwayTeam']
    
    ratings_df = fifa_ratings_df[(fifa_ratings_df['Team'] == awayteam) & (fifa_ratings_df['Date'] < date)]
    ratings_row = ratings_df.sort_values(by = ['Date', 'Version'], ascending=[False, False]).iloc[0, :]
    
    row['ADef'] = ratings_row['Defense']
    row['AMid'] = ratings_row['Midfield']
    row['AAtt'] = ratings_row['Attack']
    
    return row

In [79]:
df = df.apply(getAwayTeamRatingIndex, axis=1)
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDays,ADays,HxG,AxG,HGDL5M,HGDL5HM,HAPL5M,HAPL5HM,AGDL5M,AGDL5AM,AAPL5M,AAPL5AM,HDef,HMid,HAtt,ADef,AMid,AAtt
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.5,8.5,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,82,84,74,75,78
1,2012-09-01,swansea,sunderland,2.0,2.0,14.0,4.0,10.0,3.0,11.0,7.0,8.0,0.0,1.0,1.0,1.0,0.0,2.20,3.3,3.4,D,NaN,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,73,73,74,75,77
2,2012-09-22,west-ham,sunderland,1.0,1.0,20.0,4.0,11.0,2.0,12.0,10.0,10.0,1.0,2.0,1.0,0.0,0.0,2.10,3.3,3.6,D,NaN,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71,73,75,74,75,77
3,2012-10-06,city,sunderland,3.0,0.0,26.0,8.0,16.0,6.0,10.0,10.0,15.0,4.0,0.0,3.0,0.0,0.0,1.29,5.5,11.0,W,NaN,3,7,NaN,NaN,3.0,NaN,1.8,NaN,1.0,NaN,1.4,NaN,81,83,85,74,75,77
4,2012-10-27,stoke,sunderland,0.0,0.0,11.0,6.0,6.0,3.0,12.0,6.0,7.0,3.0,2.0,2.0,0.0,0.0,1.95,3.3,4.2,D,NaN,7,6,NaN,NaN,-1.0,NaN,1.0,NaN,-2.0,NaN,1.2,NaN,75,76,75,74,75,77


In [80]:
df.to_csv(f'datasets/cleaned_dataset.csv', index=False)